In [2]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import imutils
from tqdm import tqdm
import math
import time
from multiprocessing import Pool, cpu_count
from pathlib import Path
from PIL import Image
from IPython.display import display
import glob
import logging

In [3]:
from util_db import search_deck, search_card
from match_scale import match_scale, match_scale_mp
from match_card import match_card, match_card_v
from util_trim import trans_raw, trans_dup, trans_group, trans_sort, trans_name, trans_type, trans_paste, trans_paste_v

In [4]:
cwd = Path.cwd()
dir_main = Path.cwd().parent
n_cpu = cpu_count()

In [5]:
# Sample to Test For
card_list_freq = ["증식의 G", "하루 우라라", "코즈믹 싸이크론",
"길항승부", "무덤의 지명자", "명왕결계파",
"무한포영", "드롤 & 로크 버드", "신의 심판",
"해피의 깃털", "원시생명체 니비루", "네가로기어 아제우스",
"졸부와 겸허의 항아리", "레드 리부트", "라이트닝 스톰",
"말살의 지명자", "이펙트 뵐러", "차원 장벽"]
#######################################################
dir_card_img = f'{dir_main}/YGO_DB/'
dir_card_list_freq = []
for card in card_list_freq:
    digit = search_card(card)
    dir_card = f'{dir_card_img}{digit}.jpg'
    dir_card_list_freq.append(dir_card)

In [6]:
def round_half_up(x):
    integer_part = int(x)
    fractional_part = x - integer_part
    if fractional_part >= 0.5:
        return integer_part + 1
    else:
        return integer_part

In [7]:
dir_full_db = f'{dir_main}/Data/full_db.txt'
df = pd.read_csv(dir_full_db, sep='\t')
def fill_na_with_code(df):
    for col in df.columns:
        if col != 'Code' and df[col].isnull().any():
            df[col].fillna(df['Code'], inplace=True)
    return df
fill_na_with_code(df)

db_digit = np.asarray(df["Digit"])
db_name_ko = np.asarray(df["Name_KO"])
db_type = np.asarray(df['Type'])

# 1-1 Test

In [ ]:
deck_sample = 0
dir_deck = search_deck(deck_sample)
#######################################################

# Find deck path by deck_code
dir_deck_img = f'{dir_main}/deck'
deck_img_list = os.listdir(dir_deck_img)
deck_code_list = [os.path.splitext(filename)[0] for filename in deck_img_list]
deck_code = '0184-004'
deck_code_ind = deck_code_list.index(deck_code)
deck_path = deck_img_list[deck_code_ind]
dir_deck = f'{dir_main}/deck/{deck_path}'

#######################################################

image = Image.open(dir_deck)

o_size = image.size
f = 1000
width, height = f, int(f*o_size[1]/o_size[0])
image = image.resize((width, height))

display(image)

# preview = cv2.imread(dir_deck)
# plt.imshow(preview)

In [ ]:
card_sample = "증식의 G"
dir_card_img = f'{dir_main}/YGO_DB/'
digit_card = search_card(card_sample)
print(digit_card)
dir_card = f'{dir_card_img}{digit_card}.jpg'

image=Image.open(dir_card)

o_size = image.size
f = 40
width, height = f, int(f*o_size[1]/o_size[0])
image = image.resize((width, height))

display(image)

# preview = cv2.imread(dir_card)
# plt.imshow(preview)

# 2. Run with frequently used cards

In [ ]:
# Parameters
card_sample = "증식의 G"
dir_card_img = f'{dir_main}/YGO_DB/'
digit_card = search_card(card_sample)
dir_card = f'{dir_card_img}{digit_card}.jpg'
print(f'Name of Card : {card_sample}')
#######################################################
# Find deck path by index
deck_sample = 0
dir_deck = search_deck(deck_sample)
#######################################################
# Find deck path by deck_code
# dir_deck_img = f'{dir_main}/deck'
# deck_img_list = os.listdir(dir_deck_img)
# deck_code_list = [os.path.splitext(filename)[0] for filename in deck_img_list]
# deck_code = '0007-001'
# deck_code_ind = deck_code_list.index(deck_code)
# deck_path = deck_img_list[deck_code_ind]
# dir_deck = f'{dir_main}/deck/{deck_path}'
#######################################################
deck_code = os.path.splitext(os.path.basename(dir_deck))[0]
print(f'Name of Deck : {deck_code}')
#######################################################
deck_width = 400
card_width = match_scale(dir_card, dir_deck, deck_width)
print(f'Width of Deck : {deck_width}')
print(f'Width of Card : {card_width}')
#######################################################
threshold = 0.75
dup_tol = math.ceil(card_width/1.95)
sort_round = math.ceil(card_width/1.95)
#######################################################
# Sample To Test For
card_list = ["증식의 G", "하루 우라라", "코즈믹 싸이크론",
"길항승부", "무덤의 지명자", "명왕결계파",
"무한포영", "드롤 & 로크 버드", "신의 심판",
"해피의 깃털", "원시생명체 니비루", "네가로기어 아제우스",
"졸부와 겸허의 항아리", "레드 리부트", "라이트닝 스톰",
"말살의 지명자", "이펙트 뵐러", "차원 장벽"]
digit_list = []
dir_card_list = []
for card in card_list:
    digit = search_card(card)
    digit_list.append(digit)
    dir_card = f'{dir_card_img}{digit}.jpg'
    dir_card_list.append(dir_card)
print(f'Number of Cards to match : {len(card_list)}')

In [ ]:
# Plotter
args = [
    [dir_card, dir_deck, deck_width, card_width, threshold]
    for dir_card in dir_card_list
]
#####################################################################
with Pool(n_cpu-1) as pool:
    result = pool.starmap(match_card, args)
pool.close()
#####################################################################
card_loc = [item[0][0] for item in result]
card_pos = [item[1][0] for item in result]
#####################################################################
# Translator
zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
dup_pt = trans_dup (zip_pt, dup_tol)
group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
sort_name = trans_name(sort_digit)
sort_type = trans_type(sort_digit)
deck_list = trans_paste(sort_name, deck_code)
deck_list_v = trans_paste_v(sort_name, sort_type, deck_code)

print("Done Translation")
print(deck_list)
print(deck_list_v)

# 3. Run with Broader Card List

In [ ]:
# Parameters
card_sample = "증식의 G"
dir_card_img = f'{dir_main}/YGO_DB/'
digit_card = search_card(card_sample)
dir_card = f'{dir_card_img}{digit_card}.jpg'
#######################################################
# Find deck path by index
# deck_sample = 0
# dir_deck = search_deck(deck_sample)
#######################################################
# Find deck path by deck_code
dir_deck_img = f'{dir_main}/deck'
deck_img_list = os.listdir(dir_deck_img)
deck_code_list = [os.path.splitext(filename)[0] for filename in deck_img_list]
deck_code = '1078-004'
deck_code_ind = deck_code_list.index(deck_code)
deck_path = deck_img_list[deck_code_ind]
dir_deck = f'{dir_main}/deck/{deck_path}'
print(f'Name of Deck : {deck_code}')
#######################################################
deck_width = 400
card_width = match_scale(dir_card, dir_deck, deck_width)
print(f'single match card width : {card_width}')
threshold = 0.75
#######################################################
card_width_list = []
for dir_card in dir_card_list_freq:
    card_width = match_scale_mp(dir_card, dir_deck, deck_width, threshold)
    if card_width != None:
        card_width_list.append(card_width)
try:
    card_width = round_half_up(sum(card_width_list) / len(card_width_list))
except ZeroDivisionError:
    card_width = round_half_up(deck_width/10.1)
    print("Zero Division Error")
# print(card_width_list)
print(card_width_list)
print(f'Width of Deck : {deck_width}')
print(f'Width of Card : {card_width}')
#######################################################
dup_tol = math.floor(card_width / 1.95)
sort_round = math.floor(card_width / 1.95)
print(f'dup tol : {dup_tol} Sort Round : {sort_round}')
#######################################################
dir_card_img = f'{dir_main}/YGO_DB/'
dir_card_db = f'{dir_main}/Data/used_db_abr.txt'
with open(dir_card_db) as file:
    digit_list = [ int(line.rstrip()) for line in file]
dir_card_list = [f"{dir_card_img}{str(digit)}.jpg" for digit in digit_list]
print(f'Number of cards to match : {len(dir_card_list)}')
#######################################################
dir_deck_img = f'{dir_main}/Deck/'
dir_deck_db = f'{dir_main}/Data/deck_db.txt'
with open(dir_deck_db) as file:
    deck_code_list = [line.rstrip() for line in file]
dir_deck_list = [f"{dir_deck_img}{str(deck_code)}.jpg" for deck_code in deck_code_list]
print(f'Number of Decks to match : {len(dir_deck_list)}')

In [ ]:
# Solver
args = [
    [dir_card, dir_deck, deck_width, card_width, threshold]
    for dir_card in dir_card_list
]
#####################################################################
with Pool(n_cpu-1) as pool:
    result = pool.starmap(match_card, args)
# pool.join()
pool.close()
#####################################################################
card_loc = [item[0][0] for item in result]
card_pos = [item[1][0] for item in result]
print("Done Solution")
#####################################################################
# Translator
zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
dup_pt = trans_dup (zip_pt, dup_tol)
group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
sort_name = trans_name(sort_digit)
sort_type = trans_type(sort_digit)
deck_list = trans_paste_v(sort_name, sort_type, deck_code)
print("Done Translation")
# #####################################################################
master_db = ["\t".join(str(x) for x in deck_list)]
dir_write = f'{dir_main}/Output/single_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_db:
        fp.write("%s\n" % item)
os.startfile(dir_write)

In [ ]:
# Export Data
package = []
for i in range(len(digit_list)):
    digit = digit_list[i]
    for j in range(len(card_loc[i])):
        coord = card_loc[i][j]
        coX = coord[0]
        coY = coord[1]
        pos = card_pos[i][j]
        ind = np.where(db_digit == int(digit))[0][0]
        name = db_name_ko[ind]
        package.append(f'{coX}, {coY}, {digit}, {pos}, {name}')
        # print(f'{coX}, {coY}, {digit}, {pos}, {name}')

dir_write = f'{dir_main}\Output\package.txt'
with open(dir_write, 'w', encoding="utf-8") as fp:
    for item in package:
        fp.write("%s\n" % item)

# 5. Pre-loading Image and Cache

### 5-1-1 Parameters

In [10]:
# Parameters
card_sample = "증식의 G"
dir_card_img = f'{dir_main}/YGO_DB/'
digit_card = search_card(card_sample)
# digit_card = 14558128
dir_card = f'{dir_card_img}{digit_card}.jpg'
#######################################################
# Find deck path by deck_code
dir_deck_img = f'{dir_main}/deck'
deck_img_list = os.listdir(dir_deck_img)
deck_code_list = [os.path.splitext(filename)[0] for filename in deck_img_list]
deck_code = '0184-004'
deck_code_ind = deck_code_list.index(deck_code)
deck_path = deck_img_list[deck_code_ind]
dir_deck = f'{dir_main}/deck/{deck_path}'
#######################################################
deck_width = 400
print(f'{deck_code} | Width of Deck         | {deck_width}')
threshold = 0.7
card_width_list = []
for dir_card in dir_card_list_freq:
    card_width = match_scale_mp(dir_card, dir_deck, deck_width, threshold)
    if card_width != None:
        card_width_list.append(card_width)
#######################################################
try:
    card_width = round_half_up(sum(card_width_list) / len(card_width_list))
    print(f'{deck_code} | Width of Card         | {card_width}')
except ZeroDivisionError:
    card_width = round_half_up(deck_width/10.1)
    print(f"{deck_code} | Width of Card; Zero Div Error | {card_width}")
#######################################################
dup_tol = math.floor(card_width / 1.95)
sort_round = math.floor(card_width / 1.95)
print(f'{deck_code} | Matchng Threshold     | {threshold}')
print(f'{deck_code} | Duplicate Tolearance  | {dup_tol}')
print(f'{deck_code} | Sorting Round         | {sort_round}')
#######################################################
dir_card_img = f'{dir_main}/YGO_DB/'
dir_card_db = f'{dir_main}/Data/used_db_abr.txt'
with open(dir_card_db) as file:
    digit_list = [ int(line.rstrip()) for line in file]
dir_card_list = [f"{dir_card_img}{str(digit)}.jpg" for digit in digit_list]
print(f'{deck_code} | Cards to match        | {len(dir_card_list)}')
#######################################################
dir_deck_img = f'{dir_main}/Deck/'
dir_deck_db = f'{dir_main}/Data/deck_db.txt'
with open(dir_deck_db) as file:
    deck_code_list = [line.rstrip() for line in file]
dir_deck_list = [f"{dir_deck_img}{str(deck_code)}.jpg" for deck_code in deck_code_list]
print(f'{deck_code} | Decks to match        | {len(dir_deck_list)}')

0184-004 | Width of Deck         | 400
0184-004 | Width of Card         | 39
0184-004 | Matchng Threshold     | 0.7
0184-004 | Duplicate Tolearance  | 20
0184-004 | Sorting Round         | 20
0184-004 | Cards to match        | 7427
0184-004 | Decks to match        | 412


### 5-1-2 Test Card Widths

In [ ]:
average_list = []
for dir_deck in dir_deck_list:
    deck_code = os.path.splitext(os.path.basename(dir_deck))[0]
    card_width_list = []
    for dir_card in dir_card_list_freq:
        card_width = match_scale_mp(dir_card, dir_deck, deck_width, threshold)
        if card_width != None:
            card_width_list.append(card_width)
    try:
        average = round_half_up(sum(card_width_list) / len(card_width_list))
        average_list.append(average)
        print(f'{deck_code} | average card width is : {average}')
    except ZeroDivisionError:
        pass
        print(f'{deck_code} | Zero Division Error')

list_average = round_half_up(sum(average_list) / len(average_list))
print(f'-- average card width for overall {list_average} --')

### 5-1-3 Preloading Image

In [ ]:
load_card_list = [imutils.resize(cv2.imread(dir_card, 0), width=card_width) for dir_card in dir_card_list]

In [ ]:
load_deck_list = [cv2.cvtColor(imutils.resize(cv2.imread(dir_deck), width=deck_width), cv2.COLOR_BGR2GRAY) for dir_deck in dir_deck_list]

### 5-1-4 Save and Load Cache

In [ ]:
cache_files = glob.glob(f'{dir_main}/Output/cache_*.npy')
cache_names = [os.path.splitext(os.path.basename(file))[0] for file in cache_files]
if f'cache_{card_width:03d}' in cache_names:
    print(f'{deck_code} | -Found cache for {card_width:03d}')
else:
    print(f'{deck_code} | -No cache for {card_width:03d}')
    # getting info
    dir_card_img = f'{dir_main}/YGO_DB/'
    dir_card_db_x = f'{dir_main}/Data/used_db_max.txt'
    with open(dir_card_db_x) as file:
        digit_list_x = [ int(line.rstrip()) for line in file]
    dir_card_list_x = [f"{dir_card_img}{str(digit)}.jpg" for digit in digit_list_x]
    # resizing
    print(f'{deck_code} | -Saving cache for {card_width:03d}')
    load_card_list = [imutils.resize(cv2.imread(dir_card, 0), width=card_width) for dir_card in dir_card_list_x]
    dict_card_list = dict(zip(digit_list_x , load_card_list))
    # saving cache
    dir_cache_card_save = f'{dir_main}/Output/cache_{card_width:03d}.npy'
    np.save(dir_cache_card_save, dict_card_list)
    print(f'{deck_code} | -Saved cache for {card_width:03d}')
# loading cache
dir_cache_card_open = f'{dir_main}/Output/cache_{card_width:03d}.npy'
cache_card_list = np.load(dir_cache_card_open, allow_pickle=True)
filter_card_list = [cache_card_list.item()[digit] for digit in digit_list]
print(f'{deck_code} | -Loaded cache for {card_width:03d}')

### 5-2 Run without Cache

In [ ]:
## Solution
#####################################################################
# deck_code = 
img_deck = cv2.cvtColor(imutils.resize(cv2.imread(dir_deck), width=deck_width), cv2.COLOR_BGR2GRAY)
#####################################################################
print(f'{deck_code} | Started Process                   | Running')
t1 = time.perf_counter()
#####################################################################
# Variables
args = [[img_card, img_deck, threshold] for img_card in load_card_list]
#####################################################################
# Solver
with Pool(n_cpu-1) as pool:
    result = pool.starmap(match_card_v, args)
pool.close()
#####################################################################
print(f'{deck_code} | -Finished Template Matching in    | { (time.perf_counter() - t1) }')
t2 = time.perf_counter()
#####################################################################
card_loc = [item[0][0] for item in result]
card_pos = [item[1][0] for item in result]
#####################################################################
# Translator
zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
dup_pt = trans_dup (zip_pt, dup_tol)
group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
sort_name = trans_name(sort_digit)
sort_type = trans_type(sort_digit)
deck_list = trans_paste_v(sort_name, sort_type, deck_code)
#####################################################################
master_db = ["\t".join(str(x) for x in deck_list)]
#####################################################################
print(f'{deck_code} | -Finished Translating in          | { (time.perf_counter() - t2) }')
print(f'{deck_code} | -Finished Process in              | { (time.perf_counter() - t1) }')
#####################################################################
dir_write = f'{dir_main}\Output\single_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_db:
        fp.write("%s\n" % item)
os.startfile(dir_write)

In [ ]:
# Multi Processing
master_db = []
for i in range(0,1):
    #####################################################################
    deck_code = deck_code_list[i]
    img_deck = load_deck_list[i]
    #####################################################################
    print(f'{deck_code} | Started Process                   | Run {i}')
    t1 = time.perf_counter()
    #####################################################################
    # Variables
    args = [[img_card, img_deck, threshold] for img_card in load_card_list]
    #####################################################################
    # Solver
    with Pool(n_cpu-1) as pool:
        result = pool.starmap(match_card_v, args)
    pool.close()
    #####################################################################
    print(f'{deck_code} | -Finished Template Matching in    | { (time.perf_counter() - t1) }')
    t2 = time.perf_counter()
    #####################################################################
    card_loc = [item[0][0] for item in result]
    card_pos = [item[1][0] for item in result]
    #####################################################################
    # Translator
    zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
    dup_pt = trans_dup (zip_pt, dup_tol)
    group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
    sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
    sort_name = trans_name(sort_digit)
    sort_type = trans_type(sort_digit)
    deck_list = trans_paste_v(sort_name, sort_type, deck_code)
    #####################################################################
    master_db.append("\t".join(str(x) for x in deck_list))
    #####################################################################
    print(f'{deck_code} | -Finished Translating in          | { (time.perf_counter() - t2) }')
    print(f'{deck_code} | -Finished Process in              | { (time.perf_counter() - t1) }')
    #####################################################################
dir_write = f'{dir_main}\Output\mp_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_db:
        fp.write("%s\n" % item)
os.startfile(dir_write)

## 5-3 Run with Cache

In [ ]:
## Solution
#####################################################################
img_deck = cv2.cvtColor(imutils.resize(cv2.imread(dir_deck), width=deck_width), cv2.COLOR_BGR2GRAY)
#####################################################################
print(f'{deck_code} | Started Process                   | Run')
t1 = time.perf_counter()
#####################################################################
# Chcek Cache
cache_files = glob.glob(f'{dir_main}/Output/cache_*.npy')
cache_names = [os.path.splitext(os.path.basename(file))[0] for file in cache_files]
#####################################################################
if f'cache_{card_width:03d}' in cache_names:
    print(f'{deck_code} | -Found cache for {card_width:03d}')
else:
    print(f'{deck_code} | -No cache for {card_width:03d}')
    # getting info
    dir_card_img = f'{dir_main}/YGO_DB/'
    dir_card_db_x = f'{dir_main}/Data/used_db_max.txt'
    with open(dir_card_db_x) as file:
        digit_list_x = [ int(line.rstrip()) for line in file]
    dir_card_list_x = [f"{dir_card_img}{str(digit)}.jpg" for digit in digit_list_x]
    # resizing
    print(f'{deck_code} | -Saving cache for {card_width:03d}')
    load_card_list = [imutils.resize(cv2.imread(dir_card, 0), width=card_width) for dir_card in dir_card_list_x]
    dict_card_list = dict(zip(digit_list_x , load_card_list))
    # saving cache
    dir_cache_card_save = f'{dir_main}/Output/cache_{card_width:03d}.npy'
    np.save(dir_cache_card_save, dict_card_list)
    print(f'{deck_code} | -Saved cache for {card_width:03d}')
#####################################################################
# loading cache
dir_cache_card_open = f'{dir_main}/Output/cache_{card_width:03d}.npy'
cache_card_list = np.load(dir_cache_card_open, allow_pickle=True)
filter_card_list = [ cache_card_list.item()[digit] for digit in digit_list ]
print(f'{deck_code} | -Loaded cache for {card_width:03d}')
#####################################################################
# Variables
args = [[img_card, img_deck, threshold] for img_card in filter_card_list]
#####################################################################
# Solver
with Pool(n_cpu-1) as pool:
    result = pool.starmap(match_card_v, args)
pool.close()
#####################################################################
print(f'{deck_code} | -Finished Template Matching in    | { (time.perf_counter() - t1) }')
t2 = time.perf_counter()
#####################################################################
card_loc = [item[0][0] for item in result]
card_pos = [item[1][0] for item in result]
#####################################################################
# Translator
zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
dup_pt = trans_dup (zip_pt, dup_tol)
group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
sort_name = trans_name(sort_digit)
sort_type = trans_type(sort_digit)
deck_list = trans_paste_v(sort_name, sort_type, deck_code)
#####################################################################
master_db = ["\t".join(str(x) for x in deck_list)]
#####################################################################
print(f'{deck_code} | -Finished Translating in          | { (time.perf_counter() - t2) }')
print(f'{deck_code} | -Finished Process in              | { (time.perf_counter() - t1) }')
#####################################################################
dir_write = f'{dir_main}\Output\single_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_db:
        fp.write("%s\n" % item)
os.startfile(dir_write)

In [ ]:
# Export Data
package = []
for i in range(len(digit_list)):
    digit = digit_list[i]
    for j in range(len(card_loc[i])):
        coord = card_loc[i][j]
        coX = coord[0]
        coY = coord[1]
        pos = card_pos[i][j]
        ind = np.where(db_digit == int(digit))[0][0]
        name = db_name_ko[ind]
        package.append(f'{coX}, {coY}, {digit}, {pos}, {name}')
        # print(f'{coX}, {coY}, {digit}, {pos}, {name}')

dir_write = f'{dir_main}\Output\package.txt'
with open(dir_write, 'w', encoding="utf-8") as fp:
    for item in package:
        fp.write("%s\n" % item)

In [ ]:
import logging
logger = logging.getLogger(__name__)

In [11]:
#1865
perf_round = 2
t0 = time.perf_counter()

master_db = []
for i in range(300,400):
    #####################################################################
    deck_code = deck_code_list[i]
    dir_deck = dir_deck_list[i]
    #####################################################################
    print('======================================================')
    print(f'{deck_code} | Started Process                   | Run {i}')
    t1 = time.perf_counter()
    #####################################################################
    img_deck = cv2.cvtColor(imutils.resize(cv2.imread(dir_deck), width=deck_width), cv2.COLOR_BGR2GRAY)
    card_width_list = []
    for dir_card in dir_card_list_freq:
        card_width = match_scale_mp(dir_card, dir_deck, deck_width, threshold)
        if card_width != None:
            card_width_list.append(card_width)
    #######################################################
    print(f'{deck_code} | -Width of Deck                    | {deck_width}')
    try:
        card_width = round_half_up(sum(card_width_list) / len(card_width_list))
        print(f'{deck_code} | -Width of Card                    | {card_width}')
    except ZeroDivisionError:
        card_width = round_half_up(deck_width/10.1)
        print(f"{deck_code} | -Width of Card; Zero Div Error    | {card_width}")
    #######################################################
    dup_tol = math.floor(card_width / 1.95)
    sort_round = math.floor(card_width / 1.95)
    #######################################################
    # Check Cache
    cache_files = glob.glob(f'{dir_main}/Output/cache_*.npy')
    cache_names = [os.path.splitext(os.path.basename(file))[0] for file in cache_files]
    if f'cache_{card_width:03d}' in cache_names:
        print(f'{deck_code} | -Found cache for {card_width:03d}              |')
    else:
        print(f'{deck_code} | -No cache for {card_width:03d}                 |')
        # getting info
        dir_card_img = f'{dir_main}/YGO_DB/'
        dir_card_db_x = f'{dir_main}/Data/used_db_max.txt'
        with open(dir_card_db_x) as file:
            digit_list_x = [ int(line.rstrip()) for line in file]
        dir_card_list_x = [f"{dir_card_img}{str(digit)}.jpg" for digit in digit_list_x]
        #####################################################################
        tcs = time.perf_counter()
        print(f'{deck_code} | -Saving cache for {card_width:03d}             |')
        #####################################################################
        # resizing
        load_card_list = [imutils.resize(cv2.imread(dir_card, 0), width=card_width) for dir_card in dir_card_list_x]
        dict_card_list = dict(zip(digit_list_x , load_card_list))
        # saving cache
        dir_cache_card_save = f'{dir_main}/Output/cache_{card_width:03d}.npy'
        np.save(dir_cache_card_save, dict_card_list)
        print(f'{deck_code} | -Saved cache for {card_width:03d}              | { round(time.perf_counter() - tcs , perf_round)}')
    #####################################################################
    # loading cache
    tcl = time.perf_counter()
    print(f'{deck_code} | -Loading cache for {card_width:03d}            |')
    dir_cache_card_open = f'{dir_main}/Output/cache_{card_width:03d}.npy'
    cache_card_list = np.load(dir_cache_card_open, allow_pickle=True)
    filter_card_list = [ cache_card_list.item()[digit] for digit in digit_list ]
    print(f'{deck_code} | -Loaded cache for {card_width:03d}             | { round(time.perf_counter() - tcl , perf_round)}')
    #####################################################################
    # Variables
    args = [[img_card, img_deck, threshold] for img_card in filter_card_list]
    #####################################################################
    print(f'{deck_code} | -Starting Templating Matching     | { round(time.perf_counter() - tcl , perf_round)}')
    # Solver
    with Pool(15) as pool:
        result = pool.starmap(match_card_v, args)
    pool.close()
    #####################################################################
    print(f"{deck_code} | -Finished Template Matching in    | { round(time.perf_counter() - t1 , perf_round)}")
    t2 = time.perf_counter()
    #####################################################################
    card_loc = [item[0][0] for item in result]
    card_pos = [item[1][0] for item in result]
    #####################################################################
    # Translator
    zip_digit, zip_pt, zip_pos = trans_raw ( card_loc, card_pos , digit_list)
    dup_pt = trans_dup (zip_pt, dup_tol)
    group_pt, group_pos, group_digit = trans_group (dup_pt, zip_pt, zip_pos, zip_digit, dup_tol)
    sort_digit = trans_sort (group_pos, group_digit , group_pt , sort_round)
    sort_name = trans_name(sort_digit)
    sort_type = trans_type(sort_digit)
    deck_list = trans_paste_v(sort_name, sort_type, deck_code)
    #####################################################################
    master_db.append("\t".join(str(x) for x in deck_list))
    #####################################################################
    print(f"{deck_code} | -Finished Translating in          | { round(time.perf_counter() - t2 , perf_round)}")
    print(f"{deck_code} | Finished Process in               | = { round(time.perf_counter() - t1 , perf_round)}")
    #####################################################################
dir_write = f'{dir_main}\Output\mp_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_db:
        fp.write("%s\n" % item)

os.startfile(dir_write)

print(f'________ | Gross Computation                 | { round(time.perf_counter() - t0 , perf_round)}')

0411-005 | Started Process                   | Run 300
0411-005 | -Width of Deck                    | 400
0411-005 | -Width of Card                    | 39
0411-005 | -Found cache for 039              |
0411-005 | -Loading cache for 039            |
0411-005 | -Loaded cache for 039             | 0.2
0411-005 | -Starting Templating Matching             | 0.21
0411-005 | -Finished Template Matching in    | 12.0
0411-005 | -Finished Translating in          | 9.69
0411-005 | Finished Process in               | = 21.68
0417-004 | Started Process                   | Run 301
0417-004 | -Width of Deck                    | 400
0417-004 | -Width of Card                    | 39
0417-004 | -Found cache for 039              |
0417-004 | -Loading cache for 039            |
0417-004 | -Loaded cache for 039             | 0.06
0417-004 | -Starting Templating Matching             | 0.06
0417-004 | -Finished Template Matching in    | 11.06
0417-004 | -Finished Translating in          | 7.13
0417-004 | Fi

## 0. Test Worker

In [ ]:
import time
for i in range(6,15):
    t = time.time()
    # Variable
    args = []
    for img_card in load_card_list:
        args.append([img_card, img_deck, threshold])
    # Solver
    with Pool(i) as pool:
        result = pool.starmap(match_card_v, args)
        pool.close()
    print( f'{i} worker : { (time.time() - t) }')

# Data

In [ ]:
dir_data = 'C:/ML_YGO/Data/get_data.txt'
with open(dir_data) as file:
    data = [line.rstrip() for line in file]
df = pd.read_csv(dir_data, sep='\t')
df

In [ ]:
game_codes_nparray = df["GameCode"].values

In [ ]:
game_codes = df["GameCode"].tolist()

In [ ]:
df["GameCode"] = df["GameCode"].str.split("-")
df["GameCode"] = df["GameCode"].str.get(0)
df = df.drop_duplicates(subset=["GameCode"])
df = df.reset_index(drop=True)
df["GameCode"] = df["GameCode"].astype(int)

In [ ]:
df["Deck"] = df["Deck"].str.split("*")

# Split the resulting lists using "/" as delimiter and then using "-" as delimiter
for i, row in df.iterrows():
    new_list = []
    for item in row["Deck"]:
        if "/" in item:
            split_item = item.split("/")
            for sub_item in split_item:
                if "-" in sub_item:
                    new_list.append(sub_item.split("-")[1])
                else:
                    new_list.append(sub_item)
        elif "-" in item:
            new_list.append(item.split("-")[1])
        else:
            new_list.append(item)
    df.at[i, "Deck"] = new_list

In [ ]:
unique_game = df['GameCode'].tolist()
unique_deck = df['Deck'].tolist()

In [ ]:
df = df.explode("Deck").reset_index(drop=True)
df

In [ ]:
dicts = {unique_game[i]: unique_deck[i] for i in range(len(unique_game))}

In [ ]:
game_codes[0]

In [ ]:
true_deck = []
for i in range(len(game_codes)):
    game = game_codes[i].split("-")
    code_01 = int(game[0])
    code_02 = int(game[1])

    try:
        deck = dicts[code_01][code_02-1]
    except:
        deck = "*error"
    true_deck.append(deck)


In [ ]:
dir_write = 'C:/ML_YGO/Output/true_deck.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in true_deck:
        fp.write("%s\n" % item)

os.startfile(dir_write)